## Importamos las librerias ## 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime as dt

In [2]:
fees_original = pd.read_csv("../project_dataset/fees_data.csv")
fees = fees_original.copy()
fees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21061 entries, 0 to 21060
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               21061 non-null  int64  
 1   cash_request_id  21057 non-null  float64
 2   type             21061 non-null  object 
 3   status           21061 non-null  object 
 4   category         2196 non-null   object 
 5   total_amount     21061 non-null  float64
 6   reason           21061 non-null  object 
 7   created_at       21061 non-null  object 
 8   updated_at       21061 non-null  object 
 9   paid_at          15531 non-null  object 
 10  from_date        7766 non-null   object 
 11  to_date          7766 non-null   object 
 12  charge_moment    21061 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 2.1+ MB


In [3]:
cash_original = pd.read_csv("../project_dataset/cash_request.csv")
cash = cash_original.copy()
cash.info()
cash.isnull().sum()

FileNotFoundError: [Errno 2] No such file or directory: './project_dataset/cash_request.csv'

In [ ]:
cash.describe()

In [ ]:
cash['created_at'] = pd.to_datetime(cash['created_at'],utc = True)
cash['user_id_new'] = cash['user_id'].fillna(cash['deleted_account_id'])
cash.info()

In [ ]:
cash['created_at_by_YM'] = cash['created_at'].dt.to_period('M')
grouping1 = cash.groupby('user_id_new')['created_at_by_YM']
cash['cohorte_month'] = cash.groupby('user_id_new')['created_at_by_YM'].transform('min')
cash.head()
cash.info()

In [ ]:
print('Estos son los nulos del df Cash:\n',cash.isnull().sum())

In [ ]:
cash.groupby("status").agg('count')

In [ ]:
cash.groupby(cash['cohorte_month'])['amount'].count()

In [ ]:
# Cash Request separado por lineas a utilizar = crsplu

crsplu = cash[['id','user_id_new','status','cohorte_month','amount']]
crsplu = crsplu[crsplu['status'].isin(['money_back'])]
print(crsplu.info())
print(crsplu.head())

In [ ]:
crsplu[crsplu['cohorte_month'].isna()]

In [ ]:
fees[fees['paid_at'].isna()].head()

In [ ]:
fees.info()

In [ ]:
fees.head()

In [ ]:
for x in fees:
  print(fees[x].value_counts())

In [ ]:
fees_modidy = fees[['id','cash_request_id','type','status','total_amount','created_at','charge_moment']]
fees_accepted = fees_modidy[fees_modidy['status']== 'accepted']
fees_accepted.info()

In [ ]:
fees_accepted[fees_accepted['created_at'].isna()]

In [ ]:
fees_accepted['created_at'] = pd.to_datetime(fees_accepted['created_at'],utc = True)
fees_accepted['created_at_fees_by_YM'] = fees_accepted['created_at'].dt.to_period('M')
grouping1 = fees_accepted.groupby('cash_request_id')['created_at_fees_by_YM']
fees_accepted['cohorte_month_fees'] = fees_accepted.groupby('cash_request_id')['created_at_fees_by_YM'].transform('min')
print(fees_accepted.info())

In [ ]:
print(crsplu.info())
print(fees_accepted.info())

In [ ]:
merge_fees_cash =  crsplu.merge(fees_accepted, right_on='cash_request_id', left_on='id',how='right')
merge_fees_cash

In [ ]:

#merge_fees_cash = merge_fees_cash[~merge_fees_cash['id_x'].isna()]
merge_fees_cash.dropna(subset=['id_x'], inplace=True)
merge_fees_cash.info()


In [ ]:
for x in merge_fees_cash:
  print(merge_fees_cash[x].value_counts())

In [ ]:
# Calcular los ingresos generados por cada cohorte
cohort_ingresos = merge_fees_cash.groupby('cohorte_month')['total_amount'].sum().reset_index()
cohort_ingresos.columns = ['cohorte_month', 'Ingresos por cohorte']

print('Estos son los ingresos generados por mes\n',cohort_ingresos)
print('\nTotal de ingresos es de ',merge_fees_cash['total_amount'].sum())

## Gráficos ##

In [ ]:
# Graficar los ingresos por cohorte
plt.figure(figsize=(12, 6))
plt.bar(cohort_ingresos['cohorte_month'].astype(str), cohort_ingresos['Ingresos por cohorte'], color='orange')
plt.xlabel('Cohorte (Mes de Primera Solicitud)')
plt.ylabel('Ingresos Generados')
plt.title('Ingresos Generados por Cohorte')
plt.xticks(rotation=60)
plt.show()

In [ ]:
#merge_fees_cash['month'] = merge_fees_cash['created_at'].dt.to_period('M')

# Agrupar por cohorte y mes para sumar los ingresos
ingreso_mensual_por_cohorte= merge_fees_cash.groupby(['cohorte_month', 'created_at_fees_by_YM'])['total_amount'].sum().reset_index()

# Pivotear los datos para el mapa de calor
pivot_income = ingreso_mensual_por_cohorte.pivot(index='cohorte_month', columns='created_at_fees_by_YM', values='total_amount')
pivot_income

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_income,cmap= 'ocean_r', fmt=".1f", cbar_kws={'label': 'Ingresos'}, annot=True,linewidths=0.5)
plt.title('Ingresos Generados por Cohorte y Mes')
plt.xlabel('Mes')
plt.ylabel('Cohorte inicial')
plt.xticks(rotation=60)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
sns.heatmap(pivot_income,annot=True,fmt='.2f',cmap='crest')